# 実行環境の作成

## Google Drive

Google Driveをマウント

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google Sheets

ライブラリのインストール

In [ ]:
!pip install --upgrade -q gspread

Google Sheetsの認証

In [ ]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Google Cloud Storage

下記コードでGCPに接続

In [ ]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

# 地域設定

## 関数

In [5]:
import json

def get_area(params={}):
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    areas = json.load(j)

    if not params:
      return areas

    else:
      for p in params.keys():
        areas = list(filter(lambda x: x[p] == params[p], areas))
      return areas

## 都道府県コード一覧

In [49]:
def prefCodes():
  params = {
      'governmentType': 'prefecture',
  }
  
  area = get_area(params)
  return [d.get('areaCode') for d in area]

In [50]:
print(prefCodes())

['01000', '02000', '03000', '04000', '05000', '06000', '07000', '08000', '09000', '10000', '11000', '12000', '13000', '14000', '15000', '16000', '17000', '18000', '19000', '20000', '21000', '22000', '23000', '24000', '25000', '26000', '27000', '28000', '29000', '30000', '31000', '32000', '33000', '34000', '35000', '36000', '37000', '38000', '39000', '40000', '41000', '42000', '43000', '44000', '45000', '46000', '47000']


## 市区町村コード一覧

政令指定都市の区は無視する

In [38]:
def city(prefCode=None):
  params = {'governmentType': 'city'}

  if prefCode is not None:
    params['prefCode'] = prefCode

  area = get_area(params)
  area = list(filter(lambda item : item['bigCityFlag'] != '1', area))
  return [d.get('areaCode') for d in area]

In [39]:
cityCodes = city()
print(cityCodes)

['01100', '01202', '01203', '01204', '01205', '01206', '01207', '01208', '01209', '01210', '01211', '01212', '01213', '01214', '01215', '01216', '01217', '01218', '01219', '01220', '01221', '01222', '01223', '01224', '01225', '01226', '01227', '01228', '01229', '01230', '01231', '01233', '01234', '01235', '01236', '01303', '01304', '01331', '01332', '01333', '01334', '01337', '01343', '01345', '01346', '01347', '01361', '01362', '01363', '01364', '01367', '01370', '01371', '01391', '01392', '01393', '01394', '01395', '01396', '01397', '01398', '01399', '01400', '01401', '01402', '01403', '01404', '01405', '01406', '01407', '01408', '01409', '01423', '01424', '01425', '01427', '01428', '01429', '01430', '01431', '01432', '01433', '01434', '01436', '01437', '01438', '01452', '01453', '01454', '01455', '01456', '01457', '01458', '01459', '01460', '01461', '01462', '01463', '01464', '01465', '01468', '01469', '01470', '01471', '01472', '01481', '01482', '01483', '01484', '01485', '01486', 

In [40]:
cityCodes = city(28)
print(cityCodes)

['28100', '28201', '28202', '28203', '28204', '28205', '28206', '28207', '28208', '28209', '28210', '28212', '28213', '28214', '28215', '28216', '28217', '28218', '28219', '28220', '28221', '28222', '28223', '28224', '28225', '28226', '28227', '28228', '28229', '28301', '28365', '28381', '28382', '28442', '28443', '28446', '28464', '28481', '28501', '28585', '28586']


# Jsonを保存する関数

In [71]:
def save_json(dic,path):
  
  # Google Driveへ保存
  drive_path = "drive/MyDrive/statistics-japan/{}".format(path)
  
  with open(drive_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)
  
  # GoogleCloudStorage
  gcs_path = "statistics-japan/{}".format(path)
  with open(gcs_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)

# estatレスポンスを取得

## 関数

In [1]:
import urllib.parse
import urllib.request
import json

def get_estat_response(params):
  # appId
  params['appId'] = '724e5b90772a3e9289f41a253e4e7e32438f4fff'
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [45]:
params = {
    'statsDataId': '0000020201',
    'cdCat01': 'A1101',
    'cdArea': '28100',
    'cdTime': '1980100000',
    }

res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-12-23T16:06:43.074+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000020201', 'NARROWING_COND': {'CODE_CAT01_SELECT': 'A1101', 'CODE_AREA_SELECT': 28100, 'CODE_TIME_SELECT': 1980100000}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 1, 'FROM_NUMBER': 1, 'TO_NUMBER': 1}, 'TABLE_INF': {'@id': '0000020201', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '市区町村データ 基礎データ（廃置分合処理済）', 'TITLE': {'@no': '0000020201', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-06-21', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 1567404, 'UPDATED_DATE': '2022-06-21', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '市区町村データ', 'TABULATION_SUB_CATEGORY1': '基礎データ（廃置分合処理済）'}, 

## times

In [43]:
def times(res):
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  return [d.get('@time') for d in VALUE]

In [44]:
times(res)

['1980100000',
 '1985100000',
 '1990100000',
 '1995100000',
 '2000100000',
 '2005100000',
 '2010100000',
 '2015100000']

# 都道府県ランキング

In [115]:
import pandas as pd

def df_prefecture(params):

  # e-statレスポンス取得
  res = get_estat_response(params)
  
  # VALUE
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  
  # format data
  def is_num(s) :
    return int(s) if int(s) else float(s)

  def format(x):
    x['@time'] = x.pop('@time')[:4]
    x['$'] = is_num(x.pop('$'))
    return x

  data = list(map(format, VALUE))


  # 年次ごとに作業
  times = list(dict.fromkeys([d.get('@time') for d in data]))
  for time in times:
    d = list(filter(lambda item : item['@time'] == time, data))

    df = pd.DataFrame(d)
    df = df.rename(columns={'@cat01': 'categoryCode', '@area': 'areaCode', '@unit': 'unit', '@time': 'time', ' $': 'value'})
    df['rankJapan'] = df.rank(numeric_only=True).astype(int)

    # print(df.rank(numeric_only=True))

    print(df)


  return times

In [117]:
params = {
    'statsDataId': '0000010101',
    'cdCat01': 'A1101',
    'cdArea': ",".join(prefCodes()),
    }

df_prefecture(params)


     @tab categoryCode areaCode unit  time         $  rankJapan
0   00001        A1101    01000    人  1975   5338206         43
1   00001        A1101    02000    人  1975   1468646         21
2   00001        A1101    03000    人  1975   1385563         19
3   00001        A1101    04000    人  1975   1955267         31
4   00001        A1101    05000    人  1975   1232481         18
5   00001        A1101    06000    人  1975   1220302         17
6   00001        A1101    07000    人  1975   1970616         32
7   00001        A1101    08000    人  1975   2342198         34
8   00001        A1101    09000    人  1975   1698003         25
9   00001        A1101    10000    人  1975   1756480         28
10  00001        A1101    11000    人  1975   4821340         41
11  00001        A1101    12000    人  1975   4149147         39
12  00001        A1101    13000    人  1975  11673554         47
13  00001        A1101    14000    人  1975   6397748         45
14  00001        A1101    15000    人  19

['1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

DataFrameを整形する関数

In [ ]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json
import pandas as pd

"""
estat-APIのレスポンスを整形してDataFrameに変換
"""

def get_estat_dataframe(params):
  res = get_estat_response(params)
  df_res = conv_estat_response_to_dataframe(res)
  df_res = format_estat_dataframe(df_res)

  return df_res

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):

  # VALUEをDataFrameに変換
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df = pd.merge(df, df_class, on=key_name, how='outer')

  # 統計情報を追加
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']
  df['statsDataId'] = TABLE_INF['@id']
  df['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df

"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):

  # 列を抽出して名前変更
  df =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]
  df = df.rename(columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'})

  # 欠損データ削除
  df = df.dropna(subset=['categoryCode'])

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df['timeCode'] = df['timeCode'].astype(str)
  df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df = pd.merge(df, df_arealist, on='areaCode', how='right')


  df = df.dropna(subset=['categoryCode'])

  # '-'を'0'に置換
  df = df.replace({'value': {'-': '0'}})
  df = df.replace({'value': {'X': '0'}})

  return df

In [ ]:
params = {
    'statsDataId': '0000010101',
    'cdCat01':'A1101',
}

df = get_estat_dataframe(params)
df

# ランキングの付与

In [ ]:
!pip install pandas-gbq

BigQueryのテーブルをDataFrameに格納

In [ ]:
import pandas as pd

def get_gbq(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'contents'  
  query = f'SELECT * FROM `{project_id}.{dataset_id}.{table_id}`'
  
  return pd.read_gbq(query, project_id, dialect='standard')

ランキングの付与

In [ ]:
"""
都道府県のランキングを付与する関数
"""
def withrank_prefecture(df_arg):

  # 年度リスト、カテゴリリストの作成
  times = list(set(df_arg['timeCode'].tolist()))
  categories = list(set(df_arg['categoryCode'].tolist()))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  for time in times:
    for category in categories:
      df = df_arg.copy()
      df = df[df['timeCode'] == time][df['categoryCode'] == category]
      
      # valueを数値に変換して、'rank'を付与
      df = df.astype({'value': float})
      df['rankJapan'] = df.rank(ascending=False)['value'].astype(int).astype(str)
      
      # 結合
      df_res = pd.concat([df_res, df])
  
  return df_res

In [ ]:
table_id = 'total-population-prefecture'
df = get_gbq(table_id)
df_rank = withrank_prefecture(df)
df_rank

In [ ]:
"""
市区町村のランキングを付与する関数
"""
def withrank_city(df_arg):
  print(f'基のDataDrameは{len(df_arg)}行')

  # 年度リスト、カテゴリリストの作成
  times = list(set(df_arg['timeCode'].tolist()))
  categories = list(set(df_arg['categoryCode'].tolist()))
  prefs = list(set(df_arg['prefCode'].tolist()))
  print(prefs)

  # ランキングDataFrameを作成
  df_rank = pd.DataFrame(index=[], columns=[])
  for time in times:
    for category in categories:

      # DataFrameの抽出
      df = df_arg.copy()
      df = df[df['timeCode'] == time][df['categoryCode'] == category]
      df = df[df['bigCityFlag'] != '1']

      # valueを数値に変換して、'rank'を付与
      df = df.astype({'value': float})
      df['rankJapan'] = df.rank(ascending=False)['value'].astype(int).astype(str)
      
      # 必要な列だけ結合
      df = df[['timeCode','categoryCode','areaCode','rankJapan']]
      df_rank = pd.concat([df_rank, df])
  
  # 基のDataFrameにマージ
  df_res = df_arg.copy()
  df_res = pd.merge(df_res, df_rank, on=['timeCode','categoryCode','areaCode'], how='left')


  # ランキングDataFrameを作成
  df_rank = pd.DataFrame(index=[], columns=[])
  for time in times:
    for category in categories:
      for pref in prefs:
        # DataFrameの抽出
        df = df_arg.copy()
        df = df[df['timeCode'] == time][df['categoryCode'] == category]
        df = df[df['bigCityFlag'] != '1']
        df = df[df['prefCode'] == pref]

        # valueを数値に変換して、'rank'を付与
        df = df.astype({'value': float})
        df['rankPref'] = df.rank(ascending=False)['value'].astype(int).astype(str)
      
        # 必要な列だけ結合
        df = df[['timeCode','categoryCode','areaCode','rankPref']]
        df_rank = pd.concat([df_rank, df])

  df_res = pd.merge(df_res, df_rank, on=['timeCode','categoryCode','areaCode'], how='left')
  return df_res

In [ ]:
table_id = 'total-population-city'
df = get_gbq(table_id)
df_rank = withrank_city(df)
df_rank

NameError: ignored

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# 統計カード一覧を取得
with open('/content/drive/MyDrive/statistics-hyogo/contents/statistics-cards.json') as j:
  cardlist = json.load(j)

for card in cardlist:

  # BigQueryのテーブル定義
  project_id='primal-buttress-342908'
  dataset_id = 'contents'
  table_id= card['cardId']

  client = bigquery.Client()

  try:
    client.get_table(f'{project_id}.{dataset_id}.{table_id}')  
    print("Table {} already exists.".format(table_id))

  except NotFound:
    estatParams = card['estatParams']
    governmentType = card['governmentType']
    # print(governmentType)

    df = get_estat_dataframe(estatParams)

    if governmentType == 'prefecture':
      df = withrank_prefecture(df)
      print(f'{table_id}のデータを作成しています')
      print(estatParams)
    else:
      df = withrank_city(df)
    
    df.to_gbq(f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")
    
    print(estatParams)
    print(f'{table_id}を保存しました')

